In [1]:
import torch
from src.models.blm.pl_training import Transformer
from src.tokenize.tokenizer import Tokenizer

MODEL_CHECKPOINT_PATH = "/home/pranav-pc/projects/model-registry/blm-medium/last.ckpt"
TOKENIZER_CHECKPOINT = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/tokenizer_checkpoints/"
)
tokenizer = Tokenizer(TOKENIZER_CHECKPOINT)
model = Transformer.load_from_checkpoint(MODEL_CHECKPOINT_PATH)
model = torch.compile(model, dynamic=True)

#### Inference
model.eval()
model = model.cuda()

import os

os.environ["WANDB_DISABLED"] = "true"


def predict(text, max_new_tokens=1024, temperature=0.8, top_k=3, conditional_break=[13, 13, 1]):
    tokens = torch.LongTensor(tokenizer.encode_as_ids(text))[:-1].to("cuda:0").view(1, -1)
    # print('tokens',ds.tokenizer.encode(text,out_type=str)[:-1])
    print(
        tokenizer.decode_ids(
            model.predict_step(
                tokens,
                None,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k,
                conditional_break=conditional_break,
            )[0].tolist()
        )
    )

Seed set to 123
Seed set to 123


In [6]:
text = "Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because it couldn’t move. Every day, it would say"
predict(text)

Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because it couldn’t move. Every day, it would say, "Please, can you help me move?" But no one ever came. 

One day, a little girl came to the pumpkin. She said, "Hello! I can help you move. I can help you move." The pumpkin was very happy. 

The little girl picked up the pumpkin and put it on the ground. She said, "Now you can move. You are free now." The pumpkin was so happy. It could move again. 

The little girl smiled and said, "You are free now. Thank you for helping me." The pumpkin was so happy that it could move. 

And that is how the little girl and the pumpkin became the best of friends!


